In [1]:
#Pip install required

!pip install datasets
!pip install transformers datasets evaluate
!pip install --upgrade torch torchvision torchaudio
!python -m spacy download en_core_web_lg
!pip install --upgrade torch torchvision torchaudio
!pip install --upgrade evaluate datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 487.4/487.4 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 10.8 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.0
    Uninstalling fsspec-2025.3.0:
      Successfully uninstalled fsspec-2025.3.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.0 requires fsspec==2025.3.0, but you have fsspec 2024.12.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is 

In [2]:
import pandas as pd
import numpy as np
import re
import string
from sklearn.model_selection import train_test_split
from datasets import load_dataset
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import nltk
import os
import seaborn as sns
from collections import Counter
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import spacy
import csv
from sklearn.manifold import TSNE
from sklearn import linear_model
import matplotlib.pyplot as plt
%matplotlib inline
from IPython.display import clear_output

nlp = spacy.load('en_core_web_lg')
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [3]:
from transformers import pipeline
import pandas as pd
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import train_test_split

# Load data
df_sample = pd.read_csv("/content/yelp_cleaned_sample.csv")

# Map to binary sentiment labels
def map_true_label_to_sentiment(label):
    if label in [3, 4]:
        return "POSITIVE"
    elif label in [0, 1]:
        return "NEGATIVE"
    else:
        return "NEUTRAL"

df_sample['true_sentiment'] = df_sample['label'].apply(map_true_label_to_sentiment)

# Filter out NEUTRAL samples
df_sample = df_sample[df_sample['true_sentiment'] != "NEUTRAL"].copy()

#Stratified train-test split
train_df, test_df = train_test_split(
    df_sample,
    test_size=0.2,
    stratify=df_sample['true_sentiment'],  #label distirbution
    random_state=42
)

# Load sentiment analysis pipeline (binary classifier)
classifier = pipeline("sentiment-analysis", model="distilbert-base-uncased-finetuned-sst-2-english")

# Predict on test data only
test_df['predicted_sentiment'] = test_df['text'].apply(
    lambda text: classifier(text, truncation=True, max_length=512)[0]['label']
)

# Evaluate
accuracy = accuracy_score(test_df['true_sentiment'], test_df['predicted_sentiment'])
print(f"\n Test Accuracy: {accuracy:.4f}")
print("\n Classification Report:")
print(classification_report(test_df['true_sentiment'], test_df['predicted_sentiment']))


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

Device set to use cpu



 Test Accuracy: 0.9071

 Classification Report:
              precision    recall  f1-score   support

    NEGATIVE       0.90      0.92      0.91       805
    POSITIVE       0.91      0.90      0.91       799

    accuracy                           0.91      1604
   macro avg       0.91      0.91      0.91      1604
weighted avg       0.91      0.91      0.91      1604

